### INST326 OOP Project 03

Himanshu Gediya, Ayana Khan, Riana Barnett

INST326 Section 0203
Group number / name if applicable
Himanshu Gediya Project03 Date Due 11/18/2024

Honor Pledge

I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity, and AI use.

Disclosures and Citations

Example: ChatGPT, response to the prompt "explain how to write a function in python. give an example. Include a citation to chat gpt for the response. include the prompt in the citation." OpenAI, August 29, 2024.

### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

In [ ]:
https://github.com/Himanshuged/INST_326_Proj3_Group21/tree/main

#### 1. A Scheduling Program
>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> Example: [INST326_Fall2024/Projects/Project03](https://github.com/sdempwolf/INST326_Fall_2024/tree/main/Projects/Project03)
>
> Edit the link code below with your information, then run this cell. Test the link! It should take you to your GitHub project repository.
> [external link text](http://url_here)

In [ ]:
# Solution - enter your code solution below
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import json


DATA_FILE = "caregiver_data.json"

def save_data(caregivers):
    data = {c.name: {"hours_worked": c.hours_worked, "availability": c.availability} for c in caregivers}
    with open(DATA_FILE, "w") as file:
        json.dump(data, file)

def load_data():
    try:
        with open(DATA_FILE, "r") as file:
            data = json.load(file)
        return [Caregiver(name, details["hours_worked"], details["availability"]) for name, details in data.items()]
    except FileNotFoundError:
        return []

#Caregiver Class 
class Caregiver:
    def __init__(self, name, hours_worked=0, availability=None):
        self.name = name
        self.hours_worked = hours_worked
        self.pay_rate = 20
        self.availability = availability or {}

    def calculate_weekly_pay(self):
        return self.hours_worked * self.pay_rate

    def set_availability(self, day, shift, status='available'):
        if day not in self.availability:
            self.availability[day] = {}
        self.availability[day][shift] = status

#Schedule
class ScheduleManager:
    def __init__(self, caregivers):
        self.caregivers = caregivers
        self.schedule = {day: {"AM": None, "PM": None} for day in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]}

    def auto_assign_shifts(self):
        for day in self.schedule:
            for shift in self.schedule[day]:
                if not self.schedule[day][shift]:
                    available_caregivers = [c for c in self.caregivers if c.availability.get(day, {}).get(shift) == 'available']
                    preferred_caregivers = [c for c in self.caregivers if c.availability.get(day, {}).get(shift) == 'preferred']
                    chosen = preferred_caregivers[0] if preferred_caregivers else (available_caregivers[0] if available_caregivers else None)
                    if chosen:
                        self.schedule[day][shift] = chosen.name
                        chosen.hours_worked += 6

    def update_availability(self, caregiver_name, day, shift, status):
        caregiver = next((c for c in self.caregivers if c.name == caregiver_name), None)
        if caregiver:
            caregiver.set_availability(day, shift, status)
            if status == "unavailable" and self.schedule[day][shift] == caregiver_name:
                self.schedule[day][shift] = None  # Remove caregiver from schedule if unavailable

#GUI
class App:
    def __init__(self, root, caregivers, schedule_manager):
        self.root = root
        self.caregivers = caregivers
        self.schedule_manager = schedule_manager

        self.root.title("Caregiver Schedule Manager")
        self.root.geometry("800x600")

        # Top Frame for Inputs
        input_frame = tk.Frame(root)
        input_frame.pack(pady=10)

        # Caregiver Selection
        self.caregiver_var = tk.StringVar()
        caregivers_names = [c.name for c in caregivers]
        self.caregiver_dropdown = ttk.Combobox(input_frame, textvariable=self.caregiver_var, values=caregivers_names, state="readonly", width=20)
        self.caregiver_dropdown.set("Select Caregiver")
        self.caregiver_dropdown.grid(row=0, column=0, padx=10)

        # Day Selection
        days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
        self.day_var = tk.StringVar(value="Monday")
        self.day_dropdown = ttk.Combobox(input_frame, textvariable=self.day_var, values=days, state="readonly", width=15)
        self.day_dropdown.grid(row=0, column=1, padx=10)

        # Shift Selection
        shifts = ["AM", "PM"]
        self.shift_var = tk.StringVar(value="AM")
        self.shift_dropdown = ttk.Combobox(input_frame, textvariable=self.shift_var, values=shifts, state="readonly", width=10)
        self.shift_dropdown.grid(row=0, column=2, padx=10)

        # Availability Selection
        self.availability_var = tk.StringVar(value="available")
        availability_options = ["preferred", "available", "unavailable"]
        self.availability_dropdown = ttk.Combobox(input_frame, textvariable=self.availability_var, values=availability_options, state="readonly", width=15)
        self.availability_dropdown.grid(row=0, column=3, padx=10)

        # Update Button
        update_button = tk.Button(input_frame, text="Update Availability", command=self.update_availability)
        update_button.grid(row=0, column=4, padx=10)

        # Bottom Frame for Schedule Display
        display_frame = tk.Frame(root)
        display_frame.pack(pady=10)

        # Generate Schedule Button
        generate_schedule_button = tk.Button(display_frame, text="Generate Schedule", command=self.generate_schedule)
        generate_schedule_button.pack(pady=5)

        # Pay Report Button
        pay_report_button = tk.Button(display_frame, text="View Pay Report", command=self.calculate_pay_report)
        pay_report_button.pack(pady=5)

        # Display Schedule
        self.schedule_display = scrolledtext.ScrolledText(display_frame, width=80, height=20)
        self.schedule_display.pack(pady=10)
        self.update_schedule_display()

    def update_availability(self):
        caregiver_name = self.caregiver_var.get()
        day = self.day_var.get()
        shift = self.shift_var.get()
        availability = self.availability_var.get()

        if caregiver_name == "Select Caregiver":
            messagebox.showerror("Error", "Please select a caregiver.")
            return

        self.schedule_manager.update_availability(caregiver_name, day, shift, availability)
        save_data(self.caregivers)
        messagebox.showinfo("Success", f"Availability updated for {caregiver_name}.")
        self.update_schedule_display()

    def generate_schedule(self):
        self.schedule_manager.auto_assign_shifts()
        save_data(self.caregivers)
        self.update_schedule_display()

    def calculate_pay_report(self):
        report = "Weekly Pay Report\n"
        report += "----------------------------------------\n"
        total_pay = 0
        for caregiver in self.caregivers:
            weekly_pay = caregiver.calculate_weekly_pay()
            report += f"{caregiver.name}: ${weekly_pay:.2f}\n"
            total_pay += weekly_pay

        report += "----------------------------------------\n"
        report += f"Total Weekly Pay: ${total_pay:.2f}\n"
        report += f"Total Monthly Pay (approx): ${total_pay * 4:.2f}\n"

        self.schedule_display.configure(state="normal")
        self.schedule_display.delete("1.0", tk.END)
        self.schedule_display.insert("1.0", report)
        self.schedule_display.configure(state="disabled")

    def update_schedule_display(self):
        self.schedule_display.configure(state="normal")
        self.schedule_display.delete("1.0", tk.END)
        for day, shifts in self.schedule_manager.schedule.items():
            self.schedule_display.insert(tk.END, f"{day}:\n")
            for shift, caregiver in shifts.items():
                self.schedule_display.insert(tk.END, f"  {shift}: {caregiver if caregiver else 'No caregiver assigned'}\n")
        self.schedule_display.configure(state="disabled")

# Run Main
if __name__ == "__main__":
    # Load from file
    caregivers = load_data() or [Caregiver(f"Caregiver {i+1}") for i in range(8)]
    schedule_manager = ScheduleManager(caregivers)

    # GUI
    root = tk.Tk()
    app = App(root, caregivers, schedule_manager)
    root.mainloop()


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS